# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
import pandas as pd
import numpy as np
import nltk
import pickle
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from sqlalchemy import create_engine
# Download necessary NLTK data
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from sklearn.datasets import make_multilabel_classification

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///etl_pipeline.db')
    #print(engine.table_names())
    df = pd.read_sql ('SELECT * FROM messageCat', engine)
    df.head(20)
    X = df['message']
    y = df.iloc[:, 4:40]
    return X, y


In [5]:
### 2. Write a tokenization function to process your text data
def tokenize(texts):
# Remove non-alphanumeric characters
    texts = re.sub(r'[^\w\s]', " " , texts.lower())
    
    # Tokenize the text
    tokens = word_tokenize(texts)
    
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lemmatize and clean each token
    clean_tokens = [lemmatizer.lemmatize(tok).strip() for tok in tokens]
    
    return clean_tokens

In [16]:

# Example usage
X, y= load_data()
for message in X[:10]:
    print(message)
    print(tokenize(message),'\n')
    

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 

Information about the National Palace-
['information', 'about', 'the', 'national', 'palace'] 

Storm at sacred heart of jesus
['storm', 'at', 'sacred', 'heart', 'of', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:

def build_model():
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(tokenizer=tokenize)),  
       ('tfidf_transformer', TfidfTransformer()),           
        ('classifier', MultiOutputClassifier(RandomForestClassifier()))  
    ])
    return pipeline

pipeline = build_model()
print(pipeline)
pipeline.get_params()

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x78ecdfb23c80>, vocabulary=None)),
  ('tfidf_transformer',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('classifier',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=

In [8]:
### 4. Train pipeline
def train_pipeline( ):
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)
    X_train = [str(x) for x in X_train]
    X_test = [str(x) for x in X_test]
    model = build_model()
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    col_names = list(y.columns.values)
    print(y_test_pred)
    print(y_train_pred)
    return y_test_pred, y_train_pred, col_names ,y_train, y_test

#y_test_pred, y_train_pred, col_names, y_train, y_test = train_pipeline()
#print(y_test_pred)
#print(y_train_pred)
#print(col_names)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:

    
def get_evaluation_metrics(actual, predicted, column_names, average='macro'):
    """
    Calculate evaluation metrics for a multi-label machine learning model.

    Args:
        actual (array-like): Ground truth labels (2D array or equivalent structure).
        predicted (array-like): Predicted labels from the model (2D array or equivalent structure).
        column_names (list of str): List of field names corresponding to each label.

    Returns:
        pd.DataFrame: A DataFrame containing Accuracy, Precision, Recall, and F1 Score for each label.
    """

    # Ensure inputs are NumPy arrays for consistent processing
    actual = np.asarray(actual)
    predicted = np.asarray(predicted)

    # Calculate metrics for each field and store in a list of dictionaries
    metrics = []
    for i, column in enumerate(column_names):
        metrics.append({
            "Field": column,
            "Accuracy": accuracy_score(actual[:, i], predicted[:, i]),
            "Precision": precision_score(actual[:, i], predicted[:, i], average=average),
            "Recall": recall_score(actual[:, i], predicted[:, i], average=average),
            "F1": f1_score(actual[:, i], predicted[:, i], average=average)
        })

    # Convert metrics into a DataFrame for better presentation
    metrics_df = pd.DataFrame(metrics).set_index("Field")
    return metrics_df

In [8]:
def main():
    X,y = load_data()
    #call pipeline
    y_test_pred, y_train_pred, col_names, y_train, y_test =  train_pipeline()
    #Train
    train_df = get_evaluation_metrics(y_train, y_train_pred, col_names, average='macro')
    print(train_df)
    #Test
    test_df_first = get_evaluation_metrics(y_test, y_test_pred, col_names, average='macro')
    print(test_df_first)
    return train_df, test_df_first
    




In [9]:
main()



[[1 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 0]]
[[1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]]
                        Accuracy        F1  Precision    Recall
Field                                                          
related                 0.985189  0.943363   0.986212  0.908842
request                 0.975283  0.953516   0.984313  0.928213
offer                   0.998857  0.929125   0.999427  0.876289
aid_related             0.974045  0.973014   0.978022  0.969160
medical_help            0.979569  0.922306   0.988391  0.874212
medical_products        0.986856  0.920154   0.992551  0.867763
search_and_rescue       0.992428  0.919889   0.994996  0.865393
security                0.994333  0.903056   0.997132  0.839623
military                0.990999  0.919905   0.995387  0.865385
child_alone             1.000000  1.0000

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(                        Accuracy        F1  Precision    Recall
 Field                                                          
 related                 0.985189  0.943363   0.986212  0.908842
 request                 0.975283  0.953516   0.984313  0.928213
 offer                   0.998857  0.929125   0.999427  0.876289
 aid_related             0.974045  0.973014   0.978022  0.969160
 medical_help            0.979569  0.922306   0.988391  0.874212
 medical_products        0.986856  0.920154   0.992551  0.867763
 search_and_rescue       0.992428  0.919889   0.994996  0.865393
 security                0.994333  0.903056   0.997132  0.839623
 military                0.990999  0.919905   0.995387  0.865385
 child_alone             1.000000  1.000000   1.000000  1.000000
 water                   0.984332  0.925762   0.991300  0.877312
 food                    0.977045  0.936107   0.985709  0.897837
 shelter                 0.979617  0.929738   0.988737  0.885558
 clothing                

In [10]:
train_df, test_df_first = main()
train_df.describe()

[[1 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]]
[[1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [2 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]]
                        Accuracy        F1  Precision    Recall
Field                                                          
related                 0.984522  0.934100   0.989068  0.892552
request                 0.977045  0.957077   0.984668  0.933960
offer                   0.998952  0.935784   0.999474  0.886598
aid_related             0.971950  0.970807   0.976443  0.966577
medical_help            0.979855  0.923535   0.988543  0.875979
medical_products        0.986808  0.919907   0.991900  0.867738
search_and_rescue       0.992523  0.920897   0.996183  0.866269
security                0.995428  0.924536   0.997684  0.870620
military                0.990285  0.912500   0.995024  0.854701
child_alone             1.000000  1.0000

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Accuracy,F1,Precision,Recall
count,36.000000,36.000000,36.000000,36.000000
mean,0.987549,0.929109,0.992129,0.883047
std,0.008713,0.019481,0.005891,0.034801
min,0.970616,0.907116,0.976443,0.844000
25%,0.981403,0.916555,0.988999,0.859281
50%,0.988308,0.922940,0.993463,0.871255
75%,0.995571,0.934107,0.996412,0.888807
max,1.000000,1.000000,1.000000,1.000000


In [11]:
test_df_first.median()
test_df_first.describe()

,Accuracy,F1,Precision,Recall
count,36.000000,36.000000,36.000000,36.000000
mean,0.924164,0.506515,0.541111,0.515249
std,0.092700,0.090578,0.108890,0.088169
min,0.563238,0.340294,0.373067,0.350562
25%,0.911857,0.489099,0.489373,0.499781
50%,0.953238,0.495022,0.497285,0.500000
75%,0.982238,0.497584,0.560439,0.503095
max,1.000000,1.000000,1.000000,1.000000


Initally i have used the  RandomForestClassifier and for new model, i am trying to use MultiOutputClassifier(AdaBoostClassifier().


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:


class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            if not sentence_list:
                print(f"Empty or invalid text: {text}")
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

# Create A New Model
def build_new_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(
            AdaBoostClassifier(
                base_estimator=DecisionTreeClassifier(max_depth=1)
            )
        ))
    ])

    # Specify parameters for grid search
    param_grid = {
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__learning_rate': [0.1, 0.5]
    }

    # Create grid search object
    cv = GridSearchCV(
        estimator=pipeline, 
        param_grid=param_grid, 
        cv=3, 
        verbose=3
    )
    
    return cv

cv = build_new_model()
print('Print CV', cv)


Print CV GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...er='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__learning_rate': [0.1, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)


Best parameters 
clf__estimator__min_samples_split=2, 
clf__estimator__n_estimators=50, 
tfidf__max_df=0.75, 
tfidf__ngram_range=(1, 1) 


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
def main():
    # Load the data (you can replace this with your actual loading function)
    X, y = make_multilabel_classification(n_samples=1000, n_features=10, n_classes=3, random_state=42)
    
    # Convert each row of X to a string (textual representation of the data)
    X_text = [" ".join(map(str, row)) for row in X]  # Convert numerical features to strings
    
    # Split the dataset into training and testing sets
    X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_text, y, test_size=0.2, random_state=42)
    
    # Now, X_train_new and X_test_new are lists of strings and should work with the vectorizer
    #print(X_train_new[:5])  # Show the first 5 rows to confirm

    # Reshape X_train_new and X_test_new if needed for your model pipeline
    X_train_new = np.array(X_train_new)
    X_test_new = np.array(X_test_new)

    # Assuming the 'build_new_model()' function and 'get_evaluation_metrics()' function are defined elsewhere
    model = build_new_model()

    # Train the model
    np.random.seed(80)
    model.fit(X_train_new, y_train_new)

    # Get predictions on the training data (or you could use X_test_new here as well)
    y_train_pred = model.predict(X_train_new)
    y_test_pred = model.predict(X_test_new)
    # Evaluate model performance (assuming you have the function implemented)
    col_names = list(range(y.shape[1]))  # Assuming y is a multi-label dataset with columns for each label
    train_df = get_evaluation_metrics(y_train_new, y_train_pred, col_names, average='macro')
    test_df  = get_evaluation_metrics(y_test_new, y_test_pred, col_names, average='macro')
    # Print the evaluation metrics
    print('Train Accuracy', train_df.describe(), '\n')
    print('Test Accuracy', test_df.describe(), '\n')

In [43]:
main()

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, score=0.23220973782771537, total=   0.4s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, score=0.25842696629213485, total=   0.4s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, score=0.18796992481203006, total=   0.4s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.22846441947565543, total=   0.6s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.26591760299625467, total=   0.6s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.20676691729323307, total=   0.6s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, score=0.19101123595505617, total=   0.4s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.5, clf__estimato

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    7.3s finished


Train Accuracy        Accuracy        F1  Precision    Recall
count  3.000000  3.000000   3.000000  3.000000
mean   0.657917  0.577969   0.671249  0.600274
std    0.029377  0.070112   0.014096  0.047565
min    0.628750  0.521944   0.655692  0.562076
25%    0.643125  0.538656   0.665289  0.573636
50%    0.657500  0.555368   0.674886  0.585196
75%    0.672500  0.605981   0.679028  0.619373
max    0.687500  0.656593   0.683170  0.653550 

Test Accuracy        Accuracy        F1  Precision    Recall
count  3.000000  3.000000   3.000000  3.000000
mean   0.596667  0.491283   0.531503  0.528499
std    0.053463  0.093349   0.079065  0.064884
min    0.535000  0.432494   0.461443  0.479269
25%    0.580000  0.437465   0.488640  0.491736
50%    0.625000  0.442435   0.515837  0.504204
75%    0.627500  0.520677   0.566532  0.553114
max    0.630000  0.598920   0.617228  0.602024 



In [ ]:
#Bwlow F1 score New Model and old model


#New Model build_new_model() :Random Forest model

Train Accuracy        Accuracy        F1  Precision    Recall
count  3.000000  3.000000   3.000000  3.000000
mean   0.995833  0.995203   0.996970  0.993590
std    0.007217  0.008309   0.005249  0.011103
min    0.987500  0.985609   0.990909  0.980769
25%    0.993750  0.992804   0.995455  0.990385
50%    1.000000  1.000000   1.000000  1.000000
75%    1.000000  1.000000   1.000000  1.000000
max    1.000000  1.000000   1.000000  1.000000 

Test Accuracy        Accuracy        F1  Precision    Recall
count  3.000000  3.000000   3.000000  3.000000
mean   0.583333  0.556671   0.559707  0.556746
std    0.057735  0.041128   0.043404  0.039406
min    0.550000  0.520000   0.521978  0.520833
25%    0.550000  0.534436   0.535989  0.535669
50%    0.550000  0.548872   0.550000  0.550505
75%    0.600000  0.575006   0.578571  0.574703
max    0.650000  0.601140   0.607143  0.598901

#Old Model build_model() :AdaBoostClassifier 
Test  
	Accuracy	F1	Precision	Recall
count	36.000000	36.000000	36.000000	36.000000
mean	0.923884	0.506563	0.536845	0.515260
std	0.093727	0.090605	0.108180	0.088072
min	0.558476	0.344810	0.374710	0.352867
25%	0.912238	0.488003	0.490330	0.499806
50%	0.954190	0.494219	0.497857	0.500000
75%	0.982238	0.497944	0.549052	0.503678
max	1.000000	1.000000	1.000000	1.000000

Train 
Accuracy	F1	Precision	Recall
count	36.000000	36.000000	36.000000	36.000000
mean	0.987800	0.930657	0.992305	0.885316
std	0.008690	0.019244	0.005914	0.034609
min	0.968711	0.904986	0.976969	0.842318
25%	0.982201	0.919542	0.989128	0.863874
50%	0.988666	0.925491	0.993500	0.875053
75%	0.994940	0.933499	0.996952	0.892159
max	1.000000	1.000000	1.000000	1.000000




### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

After multiple attempts and evaluations, the Random Forest model demonstrates better performance. The new model using AdaBoostClassifier does not show any improvement in performance. Therefore, the Random Forest model will be retained as the preferred choice. It is evident that the previous model outperforms the new approach


### 9. Export your model as a pickle file

In [19]:
best_model = build_model()
#best_model.fit(X_train, y_train) 
#export_model(best_model)
# Save the model to a file
#with open('classifier.pkl', 'wb') as model_file:
#    pickle.dump(best_model, model_file)

model_file = 'classifier.pkl'
pickle.dump(best_model, open(model_file, 'wb'))
print("Model saved successfully!")
# some time later...
 


Model saved successfully!


In [22]:
# load the model from disk
#loaded_model = pickle.load(open(model_file, 'rb'))
#loaded_model
#print('successfully loaded')
#X, y = load_data()

#call pipeline
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)
#X_train = [str(x) for x in X_train]
#X_test = [str(x) for x in X_test]
#Train
#loaded_model.fit(X_train, y_train)
#print('successfully loaded')
#y_test_pred = loaded_model.predict(X_test)
#y_train_pred = loaded_model.predict(X_train)
#col_names = list(y.columns.values)
#train_df = get_evaluation_metrics(y_train, y_train_pred, col_names, average='macro')
#print(train_df)

successfully loaded
successfully loaded
                        Accuracy        F1  Precision    Recall
Field                                                          
related                 0.986142  0.960683   0.989043  0.935906
request                 0.977617  0.958156   0.985667  0.935085
offer                   0.999048  0.942290   0.999522  0.896907
aid_related             0.972331  0.971203   0.976844  0.966969
medical_help            0.979903  0.923739   0.988569  0.876273
medical_products        0.986237  0.915815   0.992223  0.861513
search_and_rescue       0.992333  0.918569   0.996087  0.862862
security                0.995428  0.924536   0.997684  0.870620
military                0.990856  0.918438   0.995314  0.863248
child_alone             1.000000  1.000000   1.000000  1.000000
water                   0.984141  0.924859   0.990260  0.876513
food                    0.977141  0.936346   0.986246  0.897891
shelter                 0.978998  0.927463   0.987436  0.882800


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.